# Applied Data Science Capstone
## Assignments
This notebook would be available on Github under the following link (Github handle @mukulbiswas)

In [2]:
import pandas as pd
import numpy as np

print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Part 1: Scraping and Preparation of Toronto Neighbourhood Data
The notebook scrapes a suggested Wikipedia pages for Toronto neighbourhood details in it. It contains the postcode details in a tabular format. Following steps are required as per the assignment given -


## Broad approach:
1. Scrape the table from the Wikipedia page
2. Preprocess the data in using pandas


_*Using the notebook from an earlier assignment.*_

In [2]:
import pandas as pd
import numpy as np
import requests as req

In [4]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [7]:
from lxml import html

myWikiPage = req.get(wikipedia_link)
page = myWikiPage.text

# Load the page (string) in a DOM structure
dom_html = html.fromstring(page)

# Use XPath to find the first instance of <title> as a DOM element
dom_rows = dom_html.xpath('/html/body/div[3]/div[3]/div[4]/div/table[1]/tbody/tr')

print('No of rows in the table are', len(dom_rows))

No of rows in the table are 290


_*290 rows including the header. There are 289 data rows.*_

In [23]:
# Create a dataframe with the required columns
data_columns = ['PostCode','Borough','Neighbourhood']
df = pd.DataFrame(columns=data_columns)

rowcount = len(dom_rows)

# for all the rows, iterate. Skip the header row.
for x1 in range(1, rowcount):
    tr_node = dom_rows[x1]
    new_row = {}

    # for each column, iterate
    for y1 in range(0,3):
        # if the content is text, store in the dict
        if (type(tr_node[y1].text) is str):
            new_row[data_columns[y1]] = tr_node[y1].text.strip()
        
        # else if the content is another tag (anchor-tag), then read the inner text
        else:
            new_row[data_columns[y1]] = tr_node[y1][0].text.strip()
    
    # insert the dict of 3 items into the df as a new row. x1 is the index
    df.loc[x1] = new_row

print('No of rows processed and columns are', df.shape)
df.head()

No of rows processed and columns are (289, 3)


,PostCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Remove the rows where Borough is *Not assigned*.

In [24]:
# Remove all the items where both borough and neighbourhood are "Not assigned"
df_2 = df[(df['Borough']!='Not assigned')]

print('The new shape of the dataframe is', df_2.shape)
df_2.head()

The new shape of the dataframe is (212, 3)


,PostCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


77 rows are dropped.

In [25]:
# Find the indexes of the rows where the neighbourhood are not assigned

indexes = df_2[df_2['Neighbourhood']=='Not assigned'].index.values

# For those indexes, replace the name of the neighbourhood with that of the Borough
df_2.loc[indexes,'Neighbourhood'] = df_2.loc[indexes,'Borough']

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [27]:
# Check if there is any row left with neighbourhood == 'Not assigned'

len(df_2[df_2['Neighbourhood']=='Not assigned'])

0

In [29]:
# Merge rows of same Post Codes; with comma-separated Borough names
df_3 = pd.DataFrame({'Neighborhood' : df_2.groupby([ 'PostCode','Borough'])['Neighbourhood'].apply(','.join)})

print('The new shape of the dataframe is', df_3.shape)
df_3.head(10)

The new shape of the dataframe is (103, 1)


,,Neighborhood
PostCode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"


## Summary and Assumptions
The above dataframe contains the scraped and preprocessed data from Wikipedia that is indexed by PostCode with multiple neighbourhood names captured against it.

It is assumed while processing the data that -
1. The term "Not assigned" has the fixed pattern. A variation with addition whitespaces would not result the same way.
2. There are no blank cells in the wikipedia page. If existed, it should have been replaced with "Not assigned" for pre-processing.
3. The wikipedia page format does not changes. The XPath of the data-table has been obtained using a browser feature availabile in developer-mode.

The final shape of the dataframe is captured below-

In [30]:
print('The final shape of the dataframe is', df_3.shape)

The final shape of the dataframe is (103, 1)


---------------------------

# Part 2: Determination of the Coordinates of the Postcodes
This is the 2nd part of the week-3 assignment (2 marks) where lat-long is determined using geocoder for each post-code that has been scraped in the part 1.

## Approach
- Starting from the output dataframe of the part 1
- Download the geocoder data available online and read into a dataframe
- Merge the geocoder dataframe with dataframe from part 1.


In [33]:
!wget http://cocl.us/Geospatial_data

--2018-12-11 14:29:29--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.72.228|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2018-12-11 14:29:29--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|159.8.72.228|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-12-11 14:29:32--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-12-11 14:29:33--  https://ibm.ent.box.com/shared/static

In [35]:
geo = pd.read_csv('Geospatial_data')

In [41]:
geo.columns=['PostCode', 'Latitude', 'Longitude']
geo = geo.set_index('PostCode')
geo.head()

,Latitude,Longitude
PostCode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [48]:
df_4 = df_3.merge(geo, left_index =True, right_index =True, how='outer').reset_index()

df_4.head(10)

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
